In [1]:
import sys
sys.path.append('C:\\PYTHONprojects\\en')
sys.path.append('C:\\PYTHONprojects')
sys.path.append('C:\\PYTHONprojects\\utilities')
import morePVs_output as opm
import morePVs as mpv
import os
import en_utilities as um
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.lines as mlines
import shutil
import seaborn as sns

Peak Demand wrt kWp and kWh
----------------------------------------------
1) AnnualPeak Load:
* plot peak load vs kWp / kWh (average across all VBs)
1a) Box plotof peak to show dist across VBS
*



 


In [2]:
sitelist = ['G', 'H'] #['F', 'G','H','I','J']# 'I',

arrangement = 'en_pv'
fontsize=12
alpha=0.6


In [3]:
# Run this cell only once to collate peak demand data from timeseries files
for sitename in sitelist:             
    # -------------------------------
    # combine  input and result files
    # -------------------------------
    project='EN2_bat2'
    study_name='energy4b_' + sitename
    base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_4\\studies'
    op = opm.Output(base_path = base_path,
                              project = project,
                              study_name = study_name)
    self = op
    df = self.data.copy()
    df_in = self.study_parameters.copy()
    df = df.merge(df_in,left_index = True, right_index=True)
    for c in df.columns:
        if '_x' in c:
            nc = c[:-2]
            df.rename(columns={c:nc}, inplace=True)
        if '_y' in c:
            df= df.drop([c],axis=1)
    
    # ---------------
    # Organise labels
    # ---------------
    df.loc[:,'site'] = df.loc[:,'load_folder'].apply(lambda x : x[-1])

  # get pv sizes for maximum pv systems
    # -----------------------------------
    maximums={}
    pv_ref_file='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\reference\\capex_pv_lookup.csv'
    pv_ref=pd.read_csv(pv_ref_file)
    pv_ref = pv_ref.set_index('pv_cap_id')
    for i in pv_ref.index:
        if 'max' in i and 'site' in i:
            site = um.find_between(i,'_','_')
            maximums[site] = pv_ref.loc[i,'kW']

    # get kWp / unit ,kWh / unit  and % bau for each scenario
    # ------------------------------------------------------------
    df['pv_filename'].fillna('zero',inplace=True)

    for s in df.index:
        site = df.loc[s,'load_folder'][-1]
        df.loc[s,'site']=site
#         if df.loc[s,'pv_filename'] == 'zero':
#             df.loc[s,'kWp_unit'] =0
#         elif 'max' in df.loc[s,'pv_filename']:
#             df.loc[s,'kWp_unit']= maximums[site]/df.loc[s,'number_of_households']
#         else:
#             df.loc[s,'kWp_unit'] = float(df.loc[s,'pv_filename'][-9]) + float(df.loc[s,'pv_filename'][-7])/10

#         df.loc[s,'kWh_unit']= df.fillna(0).loc[s,'central_battery_capacity_kWh']/df.loc[s,'number_of_households']

        df.loc[s,'short_label']= df.loc[s,'scenario_label'][11:24]

    # get timeseries data and create dfs with peak demand
    # -----------------------------------------
    ts_path = os.path.join(base_path,project,'outputs',study_name,'timeseries')
    tslist = os.listdir(ts_path)

    process_path = os.path.join(base_path,project,'outputs',study_name,'process')
    if not os.path.exists(process_path):
        os.makedirs(process_path)

    # paths for peak demand files  
    pf = 'peak_demand.csv'
    peakFile = os.path.join(process_path, pf)
    pf10 = 'peak_demand_10.csv'
    peakFile10 = os.path.join(process_path, pf10)

    # Calc single higest demand:
    dfpeak = pd.DataFrame(index = df['short_label'], columns =[str(r).zfill(2) for r in range(1,51)])
    for tsf in tslist:
        short_label = tsf[0:10]
        vb = tsf[-6:-4]
        tsFile = os.path.join(ts_path, tsf)
        dfts = pd.read_csv(tsFile)        
        dfpeak.loc[short_label,vb] = dfts['grid_import'].max()
        
    um.df_to_csv(dfpeak,peakFile)
   

    # Calc mean of 10 higest demands:
    dfpeak10 = pd.DataFrame(index = df['short_label'], columns =[str(r).zfill(2) for r in range(1,51)])    
    for tsf in tslist:
        short_label = tsf[0:10]
        vb = tsf[-6:-4]
        tsFile = os.path.join(ts_path, tsf)
        dfts = pd.read_csv(tsFile)
        dfpeak10.loc[short_label,vb] = dfts['grid_import'].nlargest(10).mean()
    um.df_to_csv(dfpeak10,peakFile10)


In [5]:
site_tag ={'A': 'a208_f12_cp34',
 'B': 'a104_f8_cp57',
 'C': 'a34_f4_cp33',
 'D': 'a138_f9_cp44',
 'E': 'a161_f7_cp38',
 'F': 'a20_f5_cp36',
 'G': 'a44_f4_cp17',
 'H': 'a52_f3_cp26',
 'I': 'a48_f4_cp09',
 'J': 'a26_f4_cp43'}
#sitelist = ['F','G','H','I','J']

In [6]:
yrange ={ 'F': [.87,.895],
 'G': [.685,.71],
 'H': [.675,.705],
 'I': [.665,.68],
 'J': [.865,.905]}

r = [0.5, 0.92]
yrange ={ 'F': r,
 'G':r,
 'H': r,
 'I': r,
 'J': r}

In [7]:
#sitelist=['F','G','H','I','J']#'I',

# Set up plotting parameters:
# ---------------------------
alpha = 0.5
cmap =  mpl.cm.tab10

study_base= 'energy4b_'
for sitename in sitelist:             
    # -------------------------------
    # combine  input and result files
    # -------------------------------

    project='EN2_bat2'
    study_name=study_base + sitename
    base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_4\\studies'
    op = opm.Output(base_path = base_path,
                              project = project,
                              study_name = study_name)
    self = op
    df = self.data.copy()
    df_in = self.study_parameters.copy()
    df = df.merge(df_in,left_index = True, right_index=True)
    for c in df.columns:
        if '_x' in c:
            nc = c[:-2]
            df.rename(columns={c:nc}, inplace=True)
        if '_y' in c:
            df= df.drop([c],axis=1)

#     arrangement = 'en_pv'
    fontsize=12
    num_units = df['number_of_households'][0]
    # get pv sizes for maximum pv systems
    # -----------------------------------
    maximums={}
    pv_ref_file='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\reference\\capex_pv_lookup.csv'
    pv_ref=pd.read_csv(pv_ref_file)
    pv_ref = pv_ref.set_index('pv_cap_id')
    for i in pv_ref.index:
        if 'max' in i and 'site' in i:
            site = um.find_between(i,'_','_')
            maximums[site] = pv_ref.loc[i,'kW']

    # get kWp / unit ,kWh / unit  and % bau for each scenario
    # ------------------------------------------------------------
    df['pv_filename'].fillna('zero',inplace=True)

    for s in df.index:
        site = df.loc[s,'load_folder'][-1]
        df.loc[s,'site']=site
        if df.loc[s,'pv_filename'] == 'zero' or 'bau' in df.loc[s,'arrangement']:
            df.loc[s,'kWp_unit'] =0
        elif 'max' in df.loc[s,'pv_filename']:
            df.loc[s,'kWp_unit']= maximums[site]/df.loc[s,'number_of_households']
        else:
            df.loc[s,'kWp_unit'] = float(df.loc[s,'pv_filename'][-9]) + float(df.loc[s,'pv_filename'][-7])/10

        if 'btm_i' in df.loc[s, 'arrangement'] or df.loc[s, 'arrangement'] == 'bau_bat':
            df.loc[s,'kWh_unit'] = df.fillna(0).loc[s,'all_battery_capacity_kWh'] + df.fillna(0).loc[s,'cp_battery_capacity_kWh']/df.loc[s,'number_of_households']
        else:
            df.loc[s,'kWh_unit']= df.fillna(0).loc[s,'central_battery_capacity_kWh']/df.loc[s,'number_of_households']
        df.loc[s,'short_label']= df.loc[s,'scenario_label'][11:24]


    # paths for peak demand files  
    # ---------------------------
    process_path = os.path.join(base_path,project,'outputs',study_name,'process')
    pf = 'peak_demand.csv'
    peakFile = os.path.join(process_path, pf)
    pf10 = 'peak_demand_10.csv'
    peakFile10 = os.path.join(process_path, pf10)

    #  reload files here if saved previously:
    dfpeak = pd.read_csv(peakFile)
    dfpeak=dfpeak.set_index('short_label')
    dfpeak10 = pd.read_csv(peakFile10)
    dfpeak10=dfpeak10.set_index('short_label')

    df['mean_peak']= df['short_label'].apply(lambda x:dfpeak.loc[x].mean())
    df['mean_peak_10highest']= df['short_label'].apply(lambda x:dfpeak10.loc[x].mean())
    df['mean_peak_pu']= df['mean_peak']/num_units


    # Directories for plots
    # ---------------------

        
    plot_pd = os.path.join(base_path,project,'project_plots','peak_demand')
    if not os.path.exists(plot_pd):
        os.makedirs(plot_pd)
   

#     plot_pf = os.path.join(op.plot_path,'peak_frequency')
#     if not os.path.exists(plot_pf):
#         os.makedirs(plot_pf)

#     box_path = os.path.join(op.plot_path,'peak_box')
#     if not os.path.exists(box_path):
#             os.makedirs(box_path)

  

    # ONE
    # Plot average of peak across all VBs or vs kWh & kWp
    # --------------------------------------------------

    for strategy in df['central_battery_strategy'].drop_duplicates().dropna().values:
        arrangements ={ 'en_pv': ['en','en_pv'],
                      'btm_i_c': ['btm_i_c','bau_bat','bau']}
#         # --------------------------------------
#         dfen = df[df['arrangement'].isin(['en', 'en_pv'])]
#         dfx=dfen.copy().loc[dfen.copy()['central_battery_strategy']==strategy].append(
#             dfen.copy().loc[dfen.copy()['kWh_unit']==0])

        # Global plotting parameters
        # -------------------------
        linewidth = 1.5
        marker = 'o'
        markersize = 4 
        linestyles ={'en_pv':'-',
                     'btm_i_c':'--'}

        dfv = {}
        dfv2={}
        for arrangement in ['en_pv','btm_i_c']:
            arrs = arrangements[arrangement]
            dfen = df[df['arrangement'].isin(arrs)]
            dfx=dfen.copy().loc[dfen['central_battery_strategy']==strategy].append(
                    dfen.copy().loc[dfen['all_battery_strategy']==strategy]).append(
                    dfen.copy().loc[(dfen['central_battery_strategy']=='') & (dfen['all_battery_strategy']=='')]).append(
             dfen.copy().loc[dfen.copy()['kWh_unit']==0])
            # ----------------------
            # reindex and stack data
            # ----------------------
            dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
            dfu = dfx['mean_peak_pu'].unstack()
            cols =dfu.columns
            dfu.columns=cols
            dfv[arrangement]=dfu.transpose()
            dfv2[arrangement] = dfu
            cols = dfv[arrangement].columns
            cols2 = [str(round(c,1)) + ' kWh / unit' for c in cols]
            dfv[arrangement].columns = cols2
            cols = dfv2[arrangement].columns
            cols2 = [str(round(c,1)) + ' kWp / unit' for c in cols]
            dfv2[arrangement].columns = cols2
                 
                 
            #dfv.loc[0,0]=df.loc[df.arrangement =='bau','mean_peak_pu'].values[0]
                                
        # ---------------------
        # Plot peak Demand vs kWp
        # ---------------------
        fig, ax = plt.subplots()                
        for arrangement in ['en_pv','btm_i_c']:
                for c in dfv[arrangement].columns:
                    y = dfv[arrangement][c].dropna()
                    x = y.index
                    line = ax.plot(x,y,   \
                                   alpha=alpha,  c = cmap(dfv[arrangement].columns.tolist().index(c)),\
                                   linewidth = linewidth, marker = marker, markersize=markersize ,
                                   linestyle = linestyles[arrangement])
        ax.set_ylim(yrange[sitename][0],yrange[sitename][1])
        ax.set_ylabel('Mean Peak Demand (kWh /unit /30 mins)')
        ax.set_xlabel('PV kWp / unit')
        ax.grid(True)
        # Label Parameters
        # ----------------
        pt = site_tag[site]
        ps = strategy
        ax.set_title('site: '+pt+' / strategy: '+ps)
        #ax.text(1.04,0.3,pt,transform=ax.transAxes,fontsize=fontsize)
        #ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

        # Legends
        # -------
        #Legends outside box and resize chart to fit
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        # Create Legend for line types     
        lline = {}
        for a in arrangements.keys():
            lline[a] = mlines.Line2D([],[],color='black', marker = 'o',
                                  linestyle=linestyles[a],
                                  markersize=markersize, label = a)
        handles = [lline[a] for a in arrangements.keys()]
        line_legend = ax.legend(handles = handles, labels = arrangements.keys(),
                                loc='upper left', bbox_to_anchor=(1, 0.7),prop={'size': 9})
        # Add the legend manually to the current Axes.
        ax1 = plt.gca().add_artist(line_legend)
        # Create legend for colours
        cline = {}
        for c in dfv[arrangement].columns:
            cline[c] = mlines.Line2D([],[],c = cmap(dfv[arrangement].columns.tolist().index(c)) , label=c)
        handles2 = [cline[c] for c in dfv[arrangement].columns]
        ax.legend(handles = handles2, labels = dfv[arrangement].columns.tolist(),
                                       loc='upper left', bbox_to_anchor=(1, 1),prop={'size': 9})
                 
        pname = 'peak_pu_kWp'  +'_'+ 'site_'  +site + '_'+ strategy +'btmi_en.jpg'
        plotFile = os.path.join(plot_pd,pname)
        plt.savefig(plotFile,dpi=1000)
        plt.close()

        # -----------------
        # Plot Peak Demand vs kWh 
        # -----------------
        fig, ax = plt.subplots()
        for arrangement in ['en_pv','btm_i_c']:
                for c in dfv2[arrangement].columns:
                    y = dfv2[arrangement][c].dropna()
                    x = y.index
                    line = ax.plot(x,y,   \
                                   alpha=alpha,  c = cmap(dfv2[arrangement].columns.tolist().index(c)),\
                                   linewidth = linewidth, marker = marker, markersize=markersize ,
                                   linestyle = linestyles[arrangement])
       
        ax.set_ylim(yrange[sitename][0],yrange[sitename][1])
        ax.set_ylabel('Mean Peak Demand (kWh /unit /30 mins)')
        ax.set_xlabel('Storage kWh / unit')
        ax.grid(True)

        # Label Parameters
        # ----------------
        pt = site_tag[site]
        ps = strategy
        ax.set_title('site: '+pt+' / strategy: '+ps)
#         ax.text(1.04,0.3,pt,transform=ax.transAxes,fontsize=fontsize)
#         ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)
        
        # Legends
        # -------
        #Legends outside box and resize chart to fit
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        # Create Legend for line types     
        lline = {}
        for a in arrangements.keys():
            lline[a] = mlines.Line2D([],[],color='black', marker = 'o',
                                  linestyle=linestyles[a],
                                  markersize=markersize, label = a)
        handles = [lline[a] for a in arrangements.keys()]
        line_legend = ax.legend(handles = handles, labels = arrangements.keys(),
                                loc='upper left', bbox_to_anchor=(1, 0.7),prop={'size': 9})
        # Add the legend manually to the current Axes.
        ax1 = plt.gca().add_artist(line_legend)
        # Create legend for colours
        cline = {}
        for c in dfv2[arrangement].columns:
            cline[c] = mlines.Line2D([],[],c = cmap(dfv2[arrangement].columns.tolist().index(c)) , label=c)
        handles2 = [cline[c] for c in dfv2[arrangement].columns]
        ax.legend(handles = handles2, labels = dfv2[arrangement].columns.tolist(),
                                       loc='upper left', bbox_to_anchor=(1, 1),prop={'size': 9})
                  
                  
        pname = 'peak_pu_kWh'  +'_'+ 'site_' + site + '_'+ strategy +'btmi_en.jpg'
        plotFile = os.path.join(plot_pd,pname)
        plt.savefig(plotFile,dpi=1000)
        plt.close()

#         # ------------------------------
#         # Plot 3D Peak demand vs kWp and kWh 
#         # ------------------------------
#         dfplot = dfv.copy()
#         x = dfplot.columns
#         y = dfplot.index
#         X,Y = np.meshgrid(x,y)
#         Z = dfplot
#         fig = plt.figure()
#         ax = fig.add_subplot(111, projection='3d')
#         ax.plot_surface(X, Y, Z)

#         # Set Axes limits
#         # ---------------
#         ax.set_zlim(yrange[sitename][0],yrange[sitename][1])
#         # ax.set_zlim(0,50)
#         # maxz=50
#         # ax.set_ylim(0,3.2)  # kWp
#         # ax.set_xlim(0,4.2)  # kWh
#         # ax.set_xticks([0,1,2,3])
#         # ax.set_yticks([0,1,2,3,4])

#         #Set rotation angle 
#         #------------------
#         ax.view_init(azim=30)
#         #ax.view_init(elev=60)

#         # Label Axes
#         # ----------
#         ax.set_zlabel('Mean Peak Demand (kWh / unit /30 mins)', fontsize=fontsize)
#         ax.set_ylabel('PV kWp / unit',fontsize=fontsize)
#         ax.set_xlabel('Storage kWh / unit',fontsize=fontsize)

#         # Set Tick Lablels
#         # ----------------
#         ax.xaxis.set_major_locator(plt.MaxNLocator(5))
#         ax.yaxis.set_major_locator(plt.MaxNLocator(4))

#         # Label Parameters
#         # ----------------
#         pa = arrangement
#         pt = site_tag[site]
#         ps = strategy
#         ax.text2D(0,1.07,pa,transform=ax.transAxes,fontsize=fontsize)
#         ax.text2D(0,1.0,pt,transform=ax.transAxes,fontsize=fontsize)
#         ax.text2D(0,0.93,ps,transform=ax.transAxes,fontsize=fontsize)

#         pname = 'peak_pu_3D'  +'_'+ 'site_'  +site + '_'+ strategy + '_' + arrangement +'.jpg'
#         plotFile = os.path.join(plot_pd,pname)
#         plt.savefig(plotFile,dpi=1000)
#         plt.close()

#     

In [18]:
dfx

,,scenario_label,arrangement,number_of_households,load_folder,en_opex,en_capex_repayment,pv_capex_repayment,average_hh_bill$,average_hh_solar_bill$,average_hh_total$,...,all_battery_id,all_battery_strategy,all_battery_capacity_kWh,site,kWp_unit,kWh_unit,short_label,mean_peak,mean_peak_10highest,mean_peak_pu
kWh_unit,kWp_unit,,,,,,,,,,,,,,,,,,,,,
4.0,0.000000,energy4b_G_hpc026_053,bau_bat,44.0,sh_t_all_site_G,0.0,0.0,0.000000,0.000000,0.0,285.426134,...,pw_scale,ch_ed1600_s_cmax,3.32,G,0.000000,4.0,hpc026_053,31.096580,28.717570,0.706740
1.0,1.744318,energy4b_G_hpc027_055,btm_i_c,44.0,sh_t_all_site_G,0.0,0.0,18475.324136,0.000000,0.0,420.840843,...,pw_scale,ch_ed1600_s_cmax,0.83,G,1.744318,1.0,hpc027_055,29.059411,26.734021,0.660441
3.0,1.744318,energy4b_G_hpc029_059,btm_i_c,44.0,sh_t_all_site_G,0.0,0.0,18475.324136,0.000000,0.0,563.553910,...,pw_scale,ch_ed1600_s_cmax,2.49,G,1.744318,3.0,hpc029_059,28.320108,26.054579,0.643639
4.0,1.744318,energy4b_G_hpc030_061,btm_i_c,44.0,sh_t_all_site_G,0.0,0.0,18475.324136,0.000000,0.0,634.910444,...,pw_scale,ch_ed1600_s_cmax,3.32,G,1.744318,4.0,hpc030_061,28.305762,26.012865,0.643313
1.0,0.500000,energy4b_G_hpc031_063,btm_i_c,44.0,sh_t_all_site_G,0.0,0.0,5560.651303,0.000000,0.0,176.543342,...,pw_scale,ch_ed1600_s_cmax,0.83,G,0.500000,1.0,hpc031_063,29.789695,27.359135,0.677039
2.0,0.500000,energy4b_G_hpc032_065,btm_i_c,44.0,sh_t_all_site_G,0.0,0.0,5560.651303,0.000000,0.0,247.899875,...,pw_scale,ch_ed1600_s_cmax,1.66,G,0.500000,2.0,hpc032_065,29.652819,27.191674,0.673928
4.0,0.500000,energy4b_G_hpc034_069,btm_i_c,44.0,sh_t_all_site_G,0.0,0.0,5560.651303,0.000000,0.0,390.612942,...,pw_scale,ch_ed1600_s_cmax,3.32,G,0.500000,4.0,hpc034_069,29.626769,27.145215,0.673336
3.0,1.000000,energy4b_G_hpc037_075,btm_i_c,44.0,sh_t_all_site_G,0.0,0.0,11121.302607,0.000000,0.0,424.443217,...,pw_scale,ch_ed1600_s_cmax,2.49,G,1.000000,3.0,hpc037_075,28.877460,26.464609,0.656306
4.0,1.000000,energy4b_G_hpc039_077,btm_i_c,44.0,sh_t_all_site_G,0.0,0.0,11121.302607,0.000000,0.0,495.799751,...,pw_scale,ch_ed1600_s_cmax,3.32,G,1.000000,4.0,hpc039_077,28.783186,26.408075,0.654163


In [21]:
df_in.loc[70:75]

,pv_filename,load_folder,arrangement,pv_cap_id,cp,all_residents,parent,network_tariff,en_capex_id,a_term,...,central_battery_id,central_battery_strategy,central_battery_capacity_kWh,battery_capex_per_kWh,cp_battery_id,cp_battery_strategy,cp_battery_capacity_kWh,all_battery_id,all_battery_strategy,all_battery_capacity_kWh
scenario,,,,,,,,,,,,,,,,,,,,,
70,site_G_1_0kw.csv,sh_t_all_site_G,btm_i_c,site_G_1_0kw_i,TIDNULL,TIDNULL,TIDNULL,EA305,capex_med,20.0,...,NaN,NaN,NaN,1000.0,pw_scale,ch_ed1700_s_cmax,7.48,pw_scale,ch_ed1700_s_cmax,0.83
71,site_G_1_0kw.csv,sh_t_all_site_G,btm_i_c,site_G_1_0kw_i,TIDNULL,TIDNULL,TIDNULL,EA305,capex_med,20.0,...,NaN,NaN,NaN,1000.0,pw_scale,ch_ed1600_s_cmax,7.48,pw_scale,ch_ed1600_s_cmax,0.83
72,site_G_1_0kw.csv,sh_t_all_site_G,btm_i_c,site_G_1_0kw_i,TIDNULL,TIDNULL,TIDNULL,EA305,capex_med,20.0,...,NaN,NaN,NaN,1000.0,pw_scale,ch_ed1700_s_cmax,14.96,pw_scale,ch_ed1700_s_cmax,1.66
73,site_G_1_0kw.csv,sh_t_all_site_G,btm_i_c,site_G_1_0kw_i,TIDNULL,TIDNULL,TIDNULL,EA305,capex_med,20.0,...,NaN,NaN,NaN,1000.0,pw_scale,ch_ed1600_s_cmax,14.96,pw_scale,ch_ed1600_s_cmax,1.66
74,site_G_1_0kw.csv,sh_t_all_site_G,btm_i_c,site_G_1_0kw_i,TIDNULL,TIDNULL,TIDNULL,EA305,capex_med,20.0,...,NaN,NaN,NaN,1000.0,pw_scale,ch_ed1700_s_cmax,22.44,pw_scale,ch_ed1700_s_cmax,2.49
75,site_G_1_0kw.csv,sh_t_all_site_G,btm_i_c,site_G_1_0kw_i,TIDNULL,TIDNULL,TIDNULL,EA305,capex_med,20.0,...,NaN,NaN,NaN,1000.0,pw_scale,ch_ed1600_s_cmax,22.44,pw_scale,ch_ed1600_s_cmax,2.49
